## Homework 3 - Machine Translation - MDS Computational Linguistics

### Assignment Topics
- Seq2seq with attention
- Evaluation metric


### Software Requirements
- Python (>=3.6)
- PyTorch (>=1.7.0) 
- Jupyter (latest)

### Submission Info.
- Due Date: March 13, 2021, 23:59:00 (Vancouver time)



## Load required packages

In [ ]:
import unicodedata
import string
import re
import random
import time
import datetime
import math

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
import torchtext
from torchtext.datasets import TranslationDataset

import spacy
import numpy as np

## Set seed of randomization and working device

In [ ]:
manual_seed = 531
torch.manual_seed(manual_seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


## Attention! Tidy Submission

rubric={mechanics:3}

To get the marks for tidy submission:
- Submit the assignment by filling in this jupyter notebook with your answers embedded.
- Be sure to follow the [general lab instructions](https://ubc-mds.github.io/resources_pages/general_lab_instructions).
- You should not use this notebook (i.e., Lab3.ipynb) to train your model on Colab. Colab may change the layout of the jupyter notebook, and then we cannot grade on your notebook, the system will miss your grade!
- We provide another jupyter notebook (i.e., `Lab3_exp.ipynb`) for you. You can load `Lab3_exp.ipynb` to Colab and run your experiments in this jupyter notebook. 
- Please download `Lab3_exp.ipynb` from Colab and include it in your final submission. 
- Some comments in your code will help us grade. Please use heading, comments, and mardown notations to organize your code. 
- Please feel free to add cells in `Lab3_exp.ipynb`.
- You don't need to submit your checkpoints.
- You can reuse any scripts of lab tutorials.

**Dataset**

In all the questions of this lab, we continue to use the English-French bilingual corpus of [Multi30k](https://github.com/multi30k/dataset) dataset that we used in lab2 and tutorials. Our task is to `translate text from French language to English language`. All your model should be trained on `train_eng_fre.tsv`, validated on `val_eng_fre.tsv`, and tested on `test_eng_fre.tsv`. 

## T3 Open-NMT translation with beam decoding
While it is completely reasonable to use PyTorch to produce models from scratch for Machine Translation, there are some tools and frameworks that can be handy, especially if you don't want to implement tricky aspects of the Machine Translation process, like efficient Beam Decoding. FAIRSeq and OpenNMT are two such frameworks, of which we will look at how to do translation with the latter.

Using the provided Open-nmt ipynb in Google Colab and the corresponding config files (multi30k.yml for T3), update the configuration to train an OpenNMT model Encoder-Decoder (w/attention) model for the Multi30k En-Fr dataset. Consider downloading your final checkpoint from colab for safekeeping (you don't need to include this in your submission).

OpenNMT_T3.ipynb should be configured to run in Colab and give an overview of how to use OpenNMT as well as sections for the T3 questions.

Copy any cells from the Colab notebook that you used to run your model, as well as any updates to the config file that you needed to make (They should be labeled T3.1-T3.4



## T3.1

### Build the vocab for the Multi30k En-Fr dataset
rubric={accuracy:2}

While just having a vocabulary is fine for some cases, using a sub-word tokenization might help capture morphological information better.


To do this, in your config file add ```transforms: [sentencepiece, filtertoolong]``` to both the training and validation corpora.

Give the code you ran to build the vocab as well as the "data" section of your multi30k config file.

Note: Use the train.fr and train-1.en files, or be prepared to split the train_eng_fre.tsv file into two separate files.

In [ ]:
# TODO build Multi30k Vocab

for file in ["train.fr","test.fr","val.fr"]:
  with open(file,"r", encoding='utf-8') as fp:
    outfile = file + ".tokd"
    with open(outfile, "w", encoding="utf-8") as out:
      for line in fp:
        tokenized = [tok.text for tok in spacy_fr.tokenizer(line)]
        out.write(" ".join(tokenized)+"\n")
for file in ["train-1.en","test.en","val.en"]:
  with open(file,"r", encoding='utf-8') as fp:
    outfile = file + ".tokd"
    with open(outfile, "w", encoding="utf-8") as out:
      for line in fp:
        tokenized = [tok.text for tok in spacy_en.tokenizer(line)]
        out.write(" ".join(tokenized)+"\n"
                  
!onmt_build_vocab -config multi30k.yml -n_sample 10000


```
Include changes you made to the Data saving, Corpus, and Vocab section in the Config HERE
````

```
# multi30k.yaml

## TO DO COMPLETE DATA SAVING
save_data: multi30k/run/model
## Where the vocab(s) will be written
src_vocab: multi30k/run/vocab.src
tgt_vocab: multi30k/run/vocab.tgt

# Corpus opts:
data:
## TODO COMPLETE CORPUS OPTIONS
## Add sentencepiece and filter long segments
    corpus_1:
        path_src: multi30k/train.fr.tokd
        path_tgt: multi30k/train-1.en.tokd
        transforms: [filtertoolong]
    valid:
        path_src: multi30k/val.fr.tokd
        path_tgt: multi30k/val.en.tokd
        #NOTE NO FILTERTOOLONG --> You don't want to bias your validation score


#TODO Fill in vocab you create (already have this above)
src_vocab: multi30k/run/vocab.src
tgt_vocab: multi30k/run/vocab.tgt
```

## T3.2
### Train Model
rubric={accuracy:2}

Fill in the multi30k.yaml config to setup a seq2seq model that has a 3 layer RNN encoder 2 layer RNN decoder, MLP attention, with 20% dropout, using Adam as your optimizer.

Copy and paste the changed parts of the *.yml file below along with the training command you used.

In [ ]:
# TODO Train Model
!onmt_train -config multi30k.yml

```
Changes to model, and optimizer here.
decoder_type: rnn
encoder_type: rnn 
enc_layers: 3
dec_layers: 2
enc_rnn_size: 500
dec_rnn_size: 500
dropout: 0.2
global_attention : mlp


# Optimizer settings
optim: adam
learning_rate: 0.001  #note changing Adam you should change the starting learning rate


```

## T3.3
### Decoding
rubric={accuracy:2}

Create predictions for the validation set using your saved models and select the one that has the highest BLEU. You should set beam size to 5 for each of these models.

Report the BLEU on this model.

In [ ]:
## Code to create predictions and calculate BLEU for models


!onmt_translate -model multi30k/run/model_step_3000.pt -src multi30k/val.fr.tokd -output multi30k/val_3000.txt -gpu 0 -beam_size 5 -seed 531 -block_ngram 2
!onmt_translate -model multi30k/run/model_step_3500.pt -src multi30k/val.fr.tokd -output multi30k/val_3500.txt -gpu 0 -beam_size 5 -seed 531 -block_ngram 2
!onmt_translate -model multi30k/run/model_step_4000.pt -src multi30k/val.fr.tokd -output multi30k/val_4000.txt -gpu 0 -beam_size 5 -seed 531 -block_ngram 2


```
BLEU = 43.32, 72.5/51.1/37.4/27.4 (BP=0.981, ratio=0.981, hyp_len=13172, ref_len=13426)
BLEU = 43.85, 73.1/51.7/38.0/28.1 (BP=0.978, ratio=0.978, hyp_len=13133, ref_len=13426)
BLEU = 43.09, 73.3/51.8/37.9/27.9 (BP=0.963, ratio=0.963, hyp_len=12933, ref_len=13426)
```

## T3.4 
### Comparing Beam Width
rubric={accuracy:3}

For your BEST model compare the peformance (Both BLEU and clocktime to run)  with the following Beam Sizes: 5 (done above), 10, 15, and 20.

Give your code and outputs below.

In [ ]:
## TODO Beam comparison
%%time

!onmt_translate -model multi30k/run/model_step_3500.pt -src multi30k/val.fr.tokd -output multi30k/beam5.txt -gpu 0 -beam_size 5 -seed 531 -block_ngram 2



```
[2021-03-16 01:59:35,899 INFO] Translating shard 0.
[2021-03-16 01:59:48,986 INFO] PRED AVG SCORE: -0.3111, PRED PPL: 1.3649
CPU times: user 35.7 ms, sys: 12.2 ms, total: 47.9 ms
Wall time: 17 s
```

In [ ]:
%%time
!onmt_translate -model multi30k/run/model_step_3500.pt -src multi30k/val.fr.tokd -output multi30k/beam10.txt -gpu 0 -beam_size 10 -seed 531 -block_ngram 2



```
[2021-03-16 01:56:26,498 INFO] Translating shard 0.
[2021-03-16 01:56:49,723 INFO] PRED AVG SCORE: -0.2985, PRED PPL: 1.3478
CPU times: user 49.2 ms, sys: 17.4 ms, total: 66.6 ms
Wall time: 27.1 s
```

In [ ]:
%%time

!onmt_translate -model multi30k/run/model_step_3500.pt -src multi30k/val.fr.tokd -output multi30k/beam15.txt -gpu 0 -beam_size 15 -seed 531 -block_ngram 2


```
[2021-03-16 01:57:22,825 INFO] Translating shard 0.
[2021-03-16 01:57:57,147 INFO] PRED AVG SCORE: -0.2971, PRED PPL: 1.3459
CPU times: user 67.3 ms, sys: 22.3 ms, total: 89.5 ms
Wall time: 38.3 s
```

In [ ]:
%%time

!onmt_translate -model multi30k/run/model_step_3500.pt -src multi30k/val.fr.tokd -output multi30k/beam20.txt -gpu 0 -beam_size 20 -seed 531 -block_ngram 2


```
[2021-03-16 01:58:01,617 INFO] Translating shard 0.
[2021-03-16 01:58:45,384 INFO] PRED AVG SCORE: -0.2973, PRED PPL: 1.3462
CPU times: user 84.8 ms, sys: 19 ms, total: 104 ms
Wall time: 47.6 s
```

Observation, beam is best at a narrow width (~5) with diminishing returns past that point. Time seems to increase ~linearly.

## Excercise 1 OpenNMT Transformer model

Also in the assignment repository is a Transformer configuration file: ```config-transformer-base-1GPU.yml``` 

The next couple questions ask about aspects of this file, with the purpose of getting you familiar with using it in case you want to use a similar configuration for the Lab 4 challenges.

### 1.1 Comparison to RNN model
rubric={reasoning:2}

Ignoring data-processing fields, what fields in the config file seem to be changed from RNN models? Pick two of these changes, look up, and explain what they do (https://opennmt.net/OpenNMT-py/options/train.html).




answer here EX:
position_encoding: 'true'
--> used for applying sinusoidal offsets to each dimension in the input, letting the model know where in the sequence each part of the input data corresponds to, to allow for parallelized translation of timesteps in the transformer model.

accum_count: 8
--> used to allow larger updates to the gradient to occur by accumulating gradient updates between batches before applying them. In essence allows for a "larger" effective batch size.





### 1.2 Transformer Config
rubric={reasoning:1}

There are a number of fields set in the config file that are obligatory or strongly recommended for running the Transformer, e.g. Glorot initialization. What are the other fields that must be set or are suggested to be set to run it correctly? (Hint you can look at https://opennmt.net/OpenNMT-py/options/train.html for a better idea of which ones are required)

answer here

heads
transformer_ff
param_init_glorot
position_encoding
max_relative_positions
self_attn_type


### 1.3 Size of Transformer
rubric={accuracy:2}

Try loading the Transformer config file and training with it (you can reuse the data processing sections from either multi30k.yml or toy-ende.yml) but set the number of steps to some trivial amount (e.g. 100). As it starts training it should load, display the model, and give a size for the number of parameters.   
EX for RNN:
```
[2021-03-02 00:42:59,826 INFO] encoder: 16506500
[2021-03-02 00:42:59,827 INFO] decoder: 41363820
[2021-03-02 00:42:59,827 INFO] * number of parameters: 57870320
```
Report how many total parameters are required to run the 'default' Transformer.

answer here
~90 million

## Exercise 2 Error analysis

Now, we have four seq2seq models: (1) basic seq2seq from week2 tutorial (`seq2seq_tutorial.ipynb`); (2) seq2seq with additive attention from week3 tutorial (`attention_tutorial.ipynb`); (3) seq2seq variant in Exercise T2.1; and finally (4) Open-NMT seq2seq with MLP attention. Please use the best models of these five models to answer the following questions. 

If you had problems creating any of these 4 models, just report your results on the models you were able to make work/develop.

### 2.1 Please report the cumulative BLEU-4 score on test set (i.e., `test_eng_fre.tsv`) via the corpus_bleu() function.
rubric={accuracy:1}

Hint: You can use `inference()` function. 

**You answer goes here:**
```
For model (1), my best model obtains 25 cumulative BLEU-4 score with 7 epoch(s).
For model (2), my best model obtains 43 cumulative BLEU-4 score with 4 epoch(s).
For model (3), my best model obtains 27 cumulative BLEU-4 score with 9 epoch(s).
For model (4), my best model obtains 43 cumulative BLEU-4 score with 3500 steps.
```

### 2.2 Please evaluate any one of four models on on test set (i.e., `test_eng_fre.tsv`) and report its BLEU-1, cumulative BLEU-2, BLEU-3, and BLEU-4 scores via the corpus_bleu() function.
rubric={accuracy:2}

Hints: 
- You can use the `inference()` function. But you will need to revise few lines of this function.
- You may need to review the BLEU implementation in week 2 tutorial. 

**You answer goes here:**
```
For Model 4 Multi-BLEU we have the following BLEU scores (Individual BLEU 1-4 components respectively):
73.1/51.7/38.0/28.1
```


```
For Model 2 we calculate the results via corpus BLEU (CUMULATIVE BLEU as opposed to individual BLEU component from Multi-BLEU):
- **This model obtains 0.72 BLEU-1 score** 
- **This model obtains 0.60 BLEU-2 score** 
- **This model obtains 0.51 BLEU-3 score** 
- **This model obtains 0.43 BLEU-4 score** 
```


### 2.3 Please explain the results of 2.2 briefly. 
rubric={reasoning:2}

Hints: What is the relationship between BLEU-n scores? Why? 

BLEU-1> BLEU-2 > BLEU-3 > BLEU-4

BLEU measures overlapping windows/sequences between source and target, and is the primary (in many works sole) measure for MT systems. 

It is harder to match larger windows/sequences exactly. 


**You answer goes here:**

To analyze the effects of sentence length, we create two subsets of test file: A. `long_test_eng_fre.tsv` which only includes sentence pairs that English reference is longer than 19 tokens; B. `short_test_eng_fre.tsv` which only includes sentence pairs that English reference is shorter than 9 tokens. `long_test_eng_fre.tsv` includes 67 sentence pairs.  `short_test_eng_fre.tsv` includes 85 sentence pairs. 

### 2.4 Please report the cumulative BLEU-4 score on long sentences of test set (i.e., `long_test_eng_fre.tsv`) via the corpus_bleu() function. 
rubric={accuracy:2}

Hint: You can use `inference()` function. 

**You answer goes here:**

Evaluate on `long_test_eng_fre.tsv`:
- For model (1), **my best model obtains 15 cumulative BLEU-4.** 
- For model (2), **my best model obtains 34 cumulative BLEU-4.**
- For model (3), **my best model obtains 16 cumulative BLEU-4.**
- For model (4), **my best model obtains 36 cumulative BLEU-4.**

### 2.5 Please report the cumulative BLEU-4 score on short sentences of test set (i.e., `short_test_eng_fre.tsv`) via the corpus_bleu() function. 
rubric={accuracy:2}

Hint: You can use `inference()` function. 

**You answer goes here:**

Evaluate on `short_test_eng_fre.tsv`:
- For model (1), **my best model obtains 31 cumulative BLEU-4.** 
- For model (2), **my best model obtains 43 cumulative BLEU-4.**
- For model (3), **my best model obtains 31 cumulative BLEU-4.**
- For model (4), **my best model obtains 44 cumulative BLEU-4.**

### 2.6 Please compare the results of questions of 2.1-2.3. Which model does perform best? What differences between their performance? How does the length of sentence affect performance?
rubric={reasoning:4}

Hint:
- You can review the Section 5 of [Luong et al. (2015)](https://arxiv.org/pdf/1508.04025.pdf) to answer this question.
- You answer should be less than 100 words. 

**Your answer goes here:**

The (4) model performs best. 
The models with Attention perform better than the models without attention. 
All the models performance do better on `short_test_eng_fre.tsv` than on the `long_test_eng_fre.tsv` set.
Without attention mechnism, the model do poorly on `long_test_eng_fre.tsv` set. 


## Exercise 3 Conceptual Questions

### 3.1 BLEU
rubric={reasoning:3}

BLEU is one of the most widely used metrics for NLP, but why can't we just use accuracy to measure the success of translations? Does accuracy make sense in machine translation? 

Two other (previously mentioned) metrics are METEOR and ROUGE. Pick one and briefly summarize how it is different than BLEU and why you might choose to use it instead of BLEU. (CITE SOURCES!)

Hint: Feel free to think back to Lab 2.

**Your answer goes here:**

Here is a good writeup of Rouge vs. BLEU https://stackoverflow.com/questions/38045290/text-summarization-evaluation-bleu-vs-rouge

With the basic idea that Rouge is better at reflecting recall of the reference, while BLEU is better at measuring precision of correct words in the generated translation.

ROUGE is often used in Summarization work as it reflects how often the summary correctly contains important parts.


https://thesai.org/Downloads/Volume6No11/Paper_28-Comparative_Study_Between_METEOR_and_BLEU_Methods.pdf

For METEOR the big difference is that it uses stems/synonyms to identify unigram overlap. Overall METEOR better reflects human judgement than BLEU.


### 3.2 Teacher Forcing
rubric={reasoning:2}

For training seq2seq models we often use Teacher Forcing as part of the training procedure. Identify two advantages of using this approach in training.  

Hint: Feel free to do some research to answer this question.

**Your answer goes here:**


Faster overall to train.

Without teacher forcing you might run into convergance issues.

### 3.3 Attention bottleneck
rubric={reasoning:1}

Learning attention functions automatically require large volumes of data especially for recurrent neural networks. Can you recommend a solution to overcome this problem? Barrett and Bingel investigate an approach to solve this limitation. Take a few minutes to SKIM the abstract and introduction and write a few sentences summarizing takeaways that you can use in practice.

Barrett, M., Bingel, J., Hollenstein, N., Rei, M., & Søgaard, A. (2018, October). Sequence classification with human attention. In Proceedings of the 22nd Conference on Computational Natural Language Learning (pp. 302-312).


**Your answer goes here:**


Using 'human-attention' can be beneficial to NLP tasks, you could potentially use open eyetracking databases (if they are on a relevant corpus) to derive these weights.

### 3.4 Global Attention Mechanisms
rubric={reasoning:1}

Global attention mechanisms consider all the encoder hidden states when deriving the context vector while local attention mechanisms consider only a subset of the encoder hidden states when deriving the context vector. [Luong et al. 2015](https://arxiv.org/pdf/1508.04025.pdf) proposed **three global attention mechanisms**. In the tutorial, we looked at (terminologies are borrowed from Luongs's paper):
* Dot: $e_{ij} = s_{i-1}^Th_j \in \mathcal{R}$
* Concat: $e_{ij} = v_a tanh( W_a [s_{i-1};h_j;]) \in \mathcal{R}$ where $W_a \in \mathcal{R}^{h\times 2h}$ and $v_a \in \mathcal{R}^{1\times h}$

Can you find out the third global attention mechanism by going over the paper (especially Section 3.1) and write it down in the same format as above?

**Your answer goes here:**

* General: $e_{ij} = s_{i-1}^TW_ah_j$ Which is basically just Dot product attention with a learned Weight matrix to modify the importance of relationships between different parts of a sentence. Dot attention is General attention with  $W_a = I$
